In [1]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 653.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install --upgrade langchain langchain_community


In [2]:
!pip install langchain sentence-transformers transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.0 MB/s eta 0:00:00


In [3]:
# Install necessary libraries

import os
import requests
from bs4 import BeautifulSoup
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import List



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Function to scrape and preprocess GitBook docs
def scrape_gitbook_docs(url: str) -> str:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.get_text()  # Get all text content
    return content

def preprocess_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()




In [5]:
# Scrape and preprocess the documentation
base_url = 'https://docs.gitbook.com/'
docs_content = scrape_gitbook_docs(base_url)
processed_content = preprocess_text(docs_content)



In [6]:
# Load Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Split content into chunks (e.g., paragraphs or sentences)
documents = processed_content.split('.')  # Example: Split by sentence
embeddings = model.encode(documents, batch_size=32, show_progress_bar=True)  # Compute embeddings for each chunk


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:

# Create a mapping of document IDs to documents
index_to_docstore_id = {i: doc for i, doc in enumerate(documents)}

# Initialize an in-memory document store
docstore = InMemoryDocstore({})

# Add documents to the in-memory document store
for i, doc in index_to_docstore_id.items():
    doc_obj = Document(page_content=doc)
    docstore._dict[i] = doc_obj  # Use the internal dictionary to store documents



In [8]:
# Define the embedding function
def embedding_function(texts: List[str]) -> np.ndarray:
    return model.encode(texts)



In [9]:
# Create a function to search for similar documents
def search_similar_documents(query_embedding: np.ndarray, embeddings: np.ndarray, k: int = 5):
    # Compute cosine similarities between the query and all document embeddings
    similarities = cosine_similarity(query_embedding, embeddings)
    # Get the top k most similar documents
    top_k_indices = np.argsort(similarities[0])[::-1][:k]
    return top_k_indices

# Load pre-trained tokenizer and model for GPT-Neo
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model_llm = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [11]:
# Ensure pad_token_id is set correctly
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token to eos token if not already set

# Define a function to generate responses with GPT-Neo
def generate_response_with_gpt_neo(input_text: str) -> str:
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,  # Adjust max length for optimization
        return_attention_mask=True
    )
    outputs = model_llm.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=100,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,  # Use temperature for more varied responses
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    formatted_response = format_response(response)
    return formatted_response


In [12]:

# Define a function to format responses
def format_response(response: str) -> str:
    response_lines = response.split('. ')  # Split sentences for line-by-line output
    formatted = "\n".join([f"- {line.strip()}" for line in response_lines if line.strip()])  # Bullet points for clarity
    return formatted

# Function to handle query and return response using Langchain and GPT-Neo
def handle_query(question: str) -> str:
    # Generate embedding for the query
    query_embedding = embedding_function([question])

    # Perform similarity search
    k = 5  # Number of closest documents to retrieve
    top_k_indices = search_similar_documents(query_embedding, embeddings, k)
    retrieved_docs = [docstore._dict[idx].page_content for idx in top_k_indices]

    # Create input text for GPT-Neo
    input_text = question + "\n" + "\n".join(retrieved_docs)

    # Generate response
    response = generate_response_with_gpt_neo(input_text)
    return response



In [13]:
# Main loop for user interaction
if __name__ == "__main__":
    while True:
        # Prompt user for input
        user_question = input("Please enter your question (or type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            print("Goodbye!")
            break

        # Generate and display response
        answer = handle_query(user_question)
        print("\nGenerated Response:\n")
        print(answer)
        print("\n" + "-"*50 + "\n")

Please enter your question (or type 'exit' to quit): What is Gitbook


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Response:

- What is Gitbook
GitBook Product DemoLast updated 1 month agoOn this pageDiscover GitBook GitBook Product DemoWas this helpful?Edit on GitHub GitBook Product Demo
Discover GitBook Content editorEdit pages, collections, content, and more in GitBook
Snippets & insightsLearn about capturing and refining knowledge in GitBook
Welcome | GitBook Documentation HomepageSolutionsAbout Community Pricing Blog Developer DocumentationMoreHomepageCommunityPricingBlogDeveloper DocumentationAsk or SearchCtrl + KWelcomeContent EditorOverviewImportEditorNavigationContent structureSpacesPagesCollectionsLive editsChange requestsEditing contentFormatting your contentInline contentMarkdownWrite with GitBook AIBlocksParagraphsHeadingsUnordered listsOrdered listsTask listsHintsQuotesCode blocksFilesImagesEmbedded URLsTablesCardsTabsExpandable sectionsDrawingsMath & TeXOpenAPI methodsSupport for CI/CD with API blocksPage linksSnippetsSearching contentGitBook AISearch & Quick findVersion c